In [2]:
import pandas as pd
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
import numpy as np
import torch

In [26]:
from transformers import BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch import nn
import torch.optim as optim
from transformers import LongformerTokenizer, LongformerForSequenceClassification

In [3]:
bert_model = BertModel.from_pretrained('bert-base-uncased')
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = bert_model
        self.dense = nn.Linear(bert_model.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids):
        bert_output = self.bert(input_ids)
        pooled_output = bert_output.pooler_output
        output = self.dense(pooled_output)
        output = self.sigmoid(output)
        return output

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aufb\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
humanset1 = pd.read_json("set1_human.json")
humanset2 = pd.read_json("set2_human.json")
machineset1 = pd.read_json("set1_machine.json")
machineset1 = machineset1.drop("machine_id",axis=1)
machineset2 = pd.read_json("set2_machine.json")
machineset2 = machineset2.drop("machine_id",axis=1)
test = pd.read_json("test.json")
test["texts"] = test["txt"].apply(lambda x:[str(each) for each in x])
test["texts"] = test["texts"].apply(lambda x:" ".join(x))

In [8]:
key = [1]*humanset1.shape[0]
humanset1["keys"] = key
key = [1]*humanset2.shape[0]
humanset2["keys"] = key
key = [0]*machineset1.shape[0]
machineset1["keys"] = key
key = [0]*machineset2.shape[0]
machineset2["keys"] = key
domain1 = pd.concat([humanset1,machineset1], axis = 0)
domain2 = pd.concat([humanset2,machineset2], axis = 0)
domain1["texts"] = domain1["txt"].apply(lambda x:[str(each) for each in x])
domain1["texts"] = domain1["texts"].apply(lambda x:" ".join(x))
domain2["texts"] = domain2["txt"].apply(lambda x:[str(each) for each in x])
domain2["texts"] = domain2["texts"].apply(lambda x:" ".join(x))

In [11]:
def pad_sequence(seq, max_length, padding_value=0):
    if len(seq) < max_length:
        return seq + [padding_value] * (max_length - len(seq))
    else:
        return seq[:max_length]
    
def filter_texts_and_keys(texts, keys, min_length=570, max_length=2775):
    filtered_texts = []
    filtered_keys = []
    for text, key in zip(texts, keys):
        length = len(text.split())
        if min_length <= length <= max_length:
            filtered_texts.append(text)
            filtered_keys.append(key)
    return filtered_texts, filtered_keys
def convert_tokenized_to_tensor(texts, max_length):
    # 将数字字符串转换为整数列表
    int_texts = [list(map(int, text.split())) for text in texts]
    # 对序列进行填充
    padded_texts = [pad_sequence(text, max_length) for text in int_texts]
    # 将填充后的序列转换为张量
    return torch.tensor(padded_texts, dtype=torch.long)
domain1_filtered_texts, domain1_filtered_keys = filter_texts_and_keys(domain1["texts"], domain1["keys"])
domain2_filtered_texts, domain2_filtered_keys = filter_texts_and_keys(domain2["texts"], domain2["keys"])

max_length = 2775

domain1_texts = convert_tokenized_to_tensor(domain1_filtered_texts, max_length)
domain1_keys = torch.tensor(domain1_filtered_keys, dtype=torch.long)

domain2_texts = convert_tokenized_to_tensor(domain2_filtered_texts, max_length)
domain2_keys = torch.tensor(domain2_filtered_keys, dtype=torch.long)


# 将数据分为训练、验证和测试集
x_train1, x_test1, y_train1, y_test1 = train_test_split(domain1_texts, domain1_keys, test_size=0.2, random_state=42)
x_train2, x_test2, y_train2, y_test2 = train_test_split(domain2_texts, domain2_keys, test_size=0.2, random_state=42)

x_train = torch.cat([x_train1, x_train2], dim=0)
y_train = torch.cat([y_train1, y_train2], dim=0)

x_test = torch.cat([x_test1, x_test2], dim=0)
y_test = torch.cat([y_test1, y_test2], dim=0)

In [12]:
class CustomDataset(Dataset):
    def __init__(self, texts, keys):
        self.texts = texts
        self.keys = keys

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.keys[idx]

train_dataset1 = CustomDataset(x_train1, y_train1)
train_dataset2 = CustomDataset(x_train2, y_train2)

test_dataset1 = CustomDataset(x_test1, y_test1)
test_dataset2 = CustomDataset(x_test2, y_test2)

train_loader1 = DataLoader(train_dataset1, batch_size=16)
train_loader2 = DataLoader(train_dataset2, batch_size=16)

test_loader1 = DataLoader(test_dataset1, batch_size=16)
test_loader2 = DataLoader(test_dataset2, batch_size=16)

In [24]:
def train_model(model, train_loader, epochs=3, device='cpu'):
    model.train()
    optimizer = optim.AdamW(model.parameters(), lr=1e-5)

    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            texts, labels = batch
            texts, labels = texts.to(device), labels.to(device)

            # 创建attention_mask
            attention_mask = (texts != 0).float()

            optimizer.zero_grad()
            outputs = model(input_ids=texts, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")


def test_model(model, test_loader, device='cpu'):
    model.eval()
    total_correct = 0
    total_count = 0

    with torch.no_grad():
        for batch in test_loader:
            texts, labels = batch
            texts, labels = texts.to(device), labels.to(device)

            # 创建attention_mask
            attention_mask = (texts != 0).float()

            outputs = model(input_ids=texts, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)

            total_correct += (predictions == labels).sum().item()
            total_count += labels.size(0)

    accuracy = total_correct / total_count
    print(f"Accuracy: {accuracy}")

In [27]:
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias'

LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (val

In [28]:
print("Training on domain1 data...")
train_model(model, train_loader1, epochs=3)
print("\nContinuing training on domain2 data...")
train_model(model, train_loader2, epochs=3)

Training on domain1 data...


RuntimeError: [enforce fail at C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1210318848 bytes.